In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
import asyncio
import operator
from typing import Optional

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langsmith import traceable
from pydantic import Field
from typing_extensions import Annotated, TypedDict

load_dotenv()

# Vanilla workflow

In [ ]:
from pydantic import BaseModel


class Evaluation(BaseModel):
    is_appropiate: bool
    explanation: str


class AggregatedResults(BaseModel):
    is_appropiate: bool
    summary: str


class State(BaseModel):
    input: str
    evaluations: Optional[list[Evaluation]] = None
    aggregated_results: Optional[AggregatedResults] = None


model = ChatOpenAI(model="gpt-4.1-mini")


@traceable
async def evaluate_text(state: State) -> Evaluation:
    model_with_str_output = model.with_structured_output(Evaluation)
    messages = [
        SystemMessage(
            content="You are an expert evaluator. Provided with a text, you will evaluate if it's appropriate for a general audience."
        ),
        HumanMessage(content=f"Evaluate the following text: {state.input}"),
    ]
    response = await model_with_str_output.ainvoke(messages)
    return response


@traceable
async def aggregate_results(state: State) -> State:
    model_with_str_output = model.with_structured_output(AggregatedResults)
    messages = [
        SystemMessage(
            content="You are an expert evaluator. Provided with a list of evaluations, you will summarize them and provide a final evaluation."
        ),
        HumanMessage(
            content=f"Summarize the following evaluations:\n\n{[(eval.explanation, eval.is_appropiate) for eval in state.evaluations]}"
        ),
    ]
    response = await model_with_str_output.ainvoke(messages)
    return response


@traceable
async def run_pipeline(input: str) -> State:
    state = State(input=input)

    evaluation_tasks = [evaluate_text(state) for _ in range(3)]
    state.evaluations = await asyncio.gather(*evaluation_tasks)

    aggregated_results = await aggregate_results(state)
    state.aggregated_results = aggregated_results
    return state


state = await run_pipeline(
    "There are athletes that consume enhancing drugs to improve their performance. For example, EPO is a drug that is used to improve performance."
)

print("Input:", state.input)
print("Individual evaluations:")
for i, eval in enumerate(state.evaluations):
    print(f"  Evaluation {i + 1}: {eval.is_appropiate} - {eval.explanation}")
print("Overall appropriate:", state.aggregated_results.is_appropiate)
print("Summarized evaluations:", state.aggregated_results.summary)

# LangGraph implementation

## Exercise
Implement a workflow that evaluates three texts in parallel and aggregates the results.